## 1. PDFからテキスト抽出

In [0]:
pdf_volume_path = '/Volumes/my_lab/mydocs_rag/raw_docs/卒論_福永敦史.pdf'
bronze_table_path = 'my_lab.mydocs_rag.mydocs_bronze'

spark.sql(f"""
    create or replace table {bronze_table_path} as
    select 
        path,
        ai_parse_document(content) as content
    from
        read_files('{pdf_volume_path}', format => 'binaryFile')
""")

## 2. ai_query関数を用いてテキストから必要情報を抽出


In [0]:
spark.sql(f"""
    create or replace table {silver_table_path} as
    select
        path,
        ai_query(
            "databricks-claude-3-7-sonnet", 
            \"\"\"
            あなたはテキスト整理のプロフェッショナルです。
            以下の卒業論文テキストから、次の項目を抽出し、JSON形式でまとめてください。

            抽出項目:
            - タイトル
            - 著者
            - 指導教員
            - 提出日
            - 所属
            - 目次（章・節名のリスト）
            - 図表目次（図・表のリスト）

            注意:
            - JSONのキーは上記項目名としてください。
            - parse_jsonできるように、余計な説明文や文字列は入れないでください。
            - 項目が見つからない場合はnullとしてください。

            例:
            {{
              \"タイトル\": \"...\",
              \"著者\": \"...\",
              \"指導教員\": \"...\",
              \"提出日\": \"...\",
              \"所属\": \"...\",
              \"目次\": [\"...\", \"...\"],
              \"図表目次\": [\"...\", \"...\"]
            }}

            卒業論文テキスト:
            {{content}}
            \"\"\",
            content
        )
    from {bronze_table_path}
""")